## 만들어야할 것들 table 5(다중회귀인 듯),6(PEIS별 주가성장),7(포트폴리오 비교)
* 각 지표별 유의성도 체크 
* 5 --> 나중에 확인 다중회귀면 가능 유효성 확인이면 개극혐
* 6 --> 그냥 수익률 상승인지아닌지에 따라서 column 하나 만들어서 .sum이랑 groupby로 하면 댈 듯
* 7 
포트폴리오 A
--> 일단 컨센별로 구분이 필요함  groupby --> 그리고 언컨센에 대해서도 할 것임 --> 여기서 한번AR,CR,N
--> PEIS별로 다시한번 groupby가 필요한 것 같은데 --> AR, CR, N
--> 마지막에 fundamental portfolio --> 그냥 뺴기 하면댐 유효성 검증은 음.. 차이 검정인거 같은데 애매하네 무슨 검정이지?
--> 근데 내생각에는 차이검정이 거의 100% 맞다 그러니깐 년도별로 평균 수익률을 구해야하는 것 같다 
포트폴리오 B
--> 그냥 똑같음 strong buy& buy는 사고 sell은 팔고 해서 수익률이 유의한지 확인하는 것임

# Table 5 만들기
1. 일단 PanelA 는 그냥 만들고
2. 나머지는 Groupby로 만들어야 함

In [6]:
import pandas as pd
import numpy as np
from Fin import *
df = pd.read_excel("Result/Compute_PEIS.xlsx")

In [2]:
consen_cut = pd.cut(df['Consensus'],bins= [-0.1, 0, 2.5, 3.5, 4.5, 5], labels = [0,1,2,3,4])
df['consen_cut'] = consen_cut

In [3]:
df_main_vars = df[['RNOA', 'deltaSGA', 'deltasale', 'deltaATO', 'GNOA', 'deltaACC', 'valPEIS','consen_cut']]

In [5]:
df_main_vars.describe()

,RNOA,deltaSGA,deltasale,deltaATO,GNOA,deltaACC,valPEIS
count,7086.000000,7.086000e+03,7077.000000,7086.000000,7.086000e+03,7086.000000,7086.000000
mean,0.642416,5.276613e+07,inf,-0.017587,3.601332e+08,0.244147,0.718036
std,35.919322,1.264109e+09,NaN,0.223833,1.848106e+09,52.205036,1.629502
min,-242.200593,-2.599364e+10,-37.665939,-4.846814,-9.668270e+08,-1176.512681,-4.000000
25%,0.010117,-1.733372e+07,-0.063528,-0.073665,1.633559e+07,-0.139593,0.000000
50%,0.088394,3.670519e+06,0.016223,-0.007808,5.989384e+07,-0.035385,1.000000
75%,0.231019,4.431493e+07,0.100040,0.042938,1.733023e+08,0.060921,2.000000
max,2788.134534,3.955632e+10,inf,3.160940,5.437252e+10,3952.877119,6.000000


In [4]:
df_mean = df_main_vars.mean()
df_std = np.sqrt(df_main_vars.var())
df_p20 = df_main_vars.quantile(0.2)
df_median = df_main_vars.median()
df_p80 = df_main_vars.quantile(0.8)
df_min = df_main_vars.min()
df_max = df_main_vars.max()
now = pd.concat([df_mean, df_std, df_p20, df_median, df_p80, df_min, df_max],axis=1)
now.columns = ['Mean','Std','P20','Median','P80','Min','Max']
now.drop('consen_cut',inplace=True)
now

,Mean,Std,P20,Median,P80,Min,Max
RNOA,6.424164e-01,3.591932e+01,-1.706264e-02,8.839358e-02,2.981004e-01,-2.422006e+02,2.788135e+03
deltaSGA,5.276613e+07,1.264109e+09,-2.981998e+07,3.670519e+06,7.059883e+07,-2.599364e+10,3.955632e+10
deltasale,inf,NaN,-9.079638e-02,1.622264e-02,1.313287e-01,-3.766594e+01,inf
deltaATO,-1.758713e-02,2.238334e-01,-9.681217e-02,-7.808189e-03,6.452850e-02,-4.846814e+00,3.160940e+00
GNOA,3.601332e+08,1.848106e+09,8.381250e+06,5.989384e+07,2.330136e+08,-9.668270e+08,5.437252e+10
deltaACC,2.441474e-01,5.220504e+01,-1.851867e-01,-3.538544e-02,1.015357e-01,-1.176513e+03,3.952877e+03
valPEIS,7.180356e-01,1.629502e+00,-1.000000e+00,1.000000e+00,2.000000e+00,-4.000000e+00,6.000000e+00


# recoommand 별 통계

In [177]:
df_recommand = df_main_vars.set_index('consen_cut')

In [226]:
def showTable5(df):
    listnum = [i for i in range(5)] *7
    listnum.sort()
    consens = df['consen_cut'].unique().sort_values()
    table5 = []
    for i in consens:
        df_mean = df[df['consen_cut'] == i ].mean()
        df_std = np.sqrt(df[df['consen_cut'] == i ].var())
        df_p20 = df[df['consen_cut'] == i ].quantile(0.2)
        df_median = df[df['consen_cut'] == i ].median()
        df_p80 = df[df['consen_cut'] == i ].quantile(0.8)
        df_min = df[df['consen_cut'] == i ].min()
        df_max = df[df['consen_cut'] == i ].max()
        now = pd.concat([df_mean, df_std, df_p20, df_median, df_p80, df_min, df_max],axis=1)
        table5.append(now)
    table5 = pd.concat(table5)
    table5.drop('consen_cut',inplace = True)
    table5.columns = ['Mean','Std','P20','Median','P80','Min','Max']
    table5.index = [ listnum, ['RNOA','deltaSGA','deltasale','deltaATO','GNOA','deltaACC','PEIS']*5]
    
    return table5

In [227]:
showTable5(df_main_vars)

Mean           Std           P20        Median  \
0 RNOA       2.707375e-01  1.698898e+01 -4.849418e-02  6.549317e-02   
  deltaSGA  -4.635122e+06  3.075146e+08 -2.265108e+07  5.101931e+05   
  deltasale           inf           NaN -1.118700e-01  2.947292e-03   
  deltaATO  -1.768375e-02  2.421575e-01 -1.061026e-01 -1.030039e-02   
  GNOA       7.182924e+07  1.266227e+08  5.687543e+06  4.006348e+07   
  deltaACC  -4.262537e-01  1.809004e+01 -2.024917e-01 -4.225567e-02   
  PEIS       1.116132e+00  1.525846e+00  0.000000e+00  1.000000e+00   
1 RNOA      -1.231895e-01  2.110049e-01 -2.910679e-01 -1.371394e-01   
  deltaSGA  -1.261484e+08  2.525709e+08 -1.369367e+08 -2.415950e+07   
  deltasale -2.181584e-01  2.499014e-01 -2.681461e-01 -9.427700e-02   
  deltaATO  -1.279496e-01  1.272297e-01 -1.845861e-01 -1.126933e-01   
  GNOA       6.495517e+07  3.424181e+07  5.291083e+07  6.694617e+07   
  deltaACC  -1.147646e-01  3.928528e-01 -4.497467e-01 -2.125548e-02   
  PEIS       2.000000e-01  1.095445e+00 -1.000000e+00  1.000000e+00   
2 RNOA      -9.930276e-01  1.478333e+01 -7.407186e-02  4.588809e-02   
  deltaSGA  -8.139713e+07  7.864749e+08 -1.677237e+08 -2.962043e+06   
  deltasale -2.750664e-02  2.090007e-01 -1.242756e-01 -1.402185e-02   
  deltaATO  -5.631284e-02  1.825011e-01 -1.229309e-01 -2.484681e-02   
  GNOA       3.142373e+08  5.979618e+08  9.002847e+06  1.273368e+08   
  deltaACC  -1.333576e+00  9.123945e+01 -1.711593e-01 -4.063819e-02   
  PEIS       2.398374e-01  1.591879e+00 -1.000000e+00  0.000000e+00   
3 RNOA       1.494925e+00  5.696434e+01  4.460669e-02  1.365173e-01   
  deltaSGA   1.721865e+08  2.108606e+09 -6.036510e+07  3.433965e+07   
  deltasale  9.918534e-02  5.011109e-01 -3.866335e-02  4.265842e-02   
  deltaATO  -1.340579e-02  1.897898e-01 -7.889354e-02 -4.079279e-03   
  GNOA       8.955346e+08  3.090569e+09  3.055893e+07  1.745675e+08   
  deltaACC   1.638928e+00  8.107321e+01 -1.490724e-01 -2.362488e-02   
  PEIS       3.697549e-02  1.579391e+00 -1.000000e+00  0.000000e+00   
4 RNOA       2.472058e-01  2.198698e-01  1.539229e-01  2.472058e-01   
  deltaSGA   3.086638e+08  3.949978e+08  1.410804e+08  3.086638e+08   
  deltasale  5.248426e-01  4.213926e-01  3.460608e-01  5.248426e-01   
  deltaATO   2.031775e-01  1.629838e-01  1.340293e-01  2.031775e-01   
  GNOA       4.044162e+08  4.737203e+08  2.034337e+08  4.044162e+08   
  deltaACC   1.810242e-01  1.156932e-01  1.319398e-01  1.810242e-01   
  PEIS      -5.000000e-01  2.121320e+00 -1.400000e+00 -5.000000e-01   

                      P80           Min           Max  
0 RNOA       2.335204e-01 -2.422006e+02  1.077710e+03  
  deltaSGA   2.586109e+07 -1.023467e+10  1.034776e+10  
  deltasale  1.159043e-01 -3.766594e+01           inf  
  deltaATO   6.924517e-02 -4.846814e+00  3.077571e+00  
  GNOA       1.034305e+08 -6.959570e+08  1.825243e+09  
  deltaACC   8.848147e-02 -1.142209e+03  2.119367e+02  
  PEIS       2.000000e+00 -4.000000e+00  6.000000e+00  
1 RNOA       7.707124e-02 -3.846308e-01  1.028810e-01  
  deltaSGA  -3.337837e+06 -5.774113e+08  2.425524e+06  
  deltasale -8.826075e-02 -6.603009e-01 -7.437430e-02  
  deltaATO  -4.382117e-02 -3.295987e-01  7.538037e-03  
  GNOA       8.167579e+07  1.218367e+07  1.072782e+08  
  deltaACC   1.040675e-01 -5.776201e-01  4.169950e-01  
  PEIS       1.000000e+00 -1.000000e+00  1.000000e+00  
2 RNOA       1.878352e-01 -2.219914e+02  1.447291e+01  
  deltaSGA   1.328895e+08 -8.158731e+09  4.509771e+09  
  deltasale  7.680862e-02 -9.082653e-01  9.478986e-01  
  deltaATO   4.338522e-02 -8.468532e-01  4.165176e-01  
  GNOA       5.293247e+08 -7.960815e+08  5.738341e+09  
  deltaACC   7.507110e-02 -1.176513e+03  8.086897e+02  
  PEIS       1.000000e+00 -4.000000e+00  5.000000e+00  
3 RNOA       4.472936e-01 -5.086974e+01  2.788135e+03  
  deltaSGA   2.629789e+08 -2.599364e+10  3.955632e+10  
  deltasale  1.576399e-01 -1.000000e+00  1.519954e+01  
  deltaATO   5.961331e-02 -2.627917e+00  3.160940e+00  
  GNOA  

In [ ]:
df_recommand.colname = 'Points'

In [ ]:
df_recommand.groupby('consen_cut').mean()

In [ ]:
df_main_vars = df_main_vars.set_index('consen_cut')

In [ ]:
dfs = yearbaseDFs(df, years = [2011,2012,2013,2014,2015,2016,2017,2018,2019,2020])

In [ ]:
df_main_vars.groupby(['consen_cut',['RNOA', 'deltaSGA', 'deltasale', 'deltaATO', 'GNOA', 'deltaACC', 'valPEIS','consen_cut']]).mean()